In [ ]:
using Catalyst, MomentClosure, Latexify

rn = @reaction_network begin
  @parameters c₁ c₂ c₃ c₄ Ω ω τ
  (c₁/Ω^2), 2X + Y → 3X
  (c₂*(1+0.5*sin(ω*(t<τ)*t))), X → Y
  (c₃*Ω, c₄), 0 ↔ X
end

raw_eqs = generate_raw_moment_eqs(rn, 2, combinatoric_ratelaws=false)
latexify(raw_eqs)

In [ ]:
println(latexify(raw_eqs))

In [ ]:
using OrdinaryDiffEq, Plots

closed_raw_eqs = moment_closure(raw_eqs, "normal")

# parameter values [c₁, c₂, c₃, c₄, Ω, ω, τ]
p = [:c₁ => 0.9, :c₂ => 2., :c₃ => 1., :c₄ => 1., :Ω => 5., :ω => 1., :τ => 40.]

# initial molecule numbers of species [X, Y]
u₀ = [1., 1.]

# deterministic initial conditions
u₀map = deterministic_IC(u₀, closed_raw_eqs)

# time interval to solve one on
tspan = (0., 100.)

# convert the closed raw moment equations into a DifferentialEquations ODEProblem
oprob = ODEProblem(closed_raw_eqs, u₀map, tspan, p)

# solve using Tsit5() solver
sol = solve(oprob, Tsit5(), saveat=0.2)
plot(sol, idxs=[1,2], lw=2)

In [ ]:
#savefig("Brusselator_time-dependent_normal.svg")

In [ ]:
using JumpProcesses

jinputs = JumpInputs(rn, u₀, tspan, p, combinatoric_ratelaws=false)
jprob = JumpProblem(jinputs, Direct())

In [ ]:
# timestep at which the solution data is saved
dt = 0.2
# the corresponding time iterator (0:0.2:100 in our case)
ts = tspan[1]:dt:tspan[2]
# save data for each trajectory only at the specified timepoints (interpolating the ODESolution)
fout = (sol, i) -> (sol(ts), false)
ensembleprob  = EnsembleProblem(jprob, output_func=fout)

# simulate 10000 SSA trajectories (can get very slow...)
@time sol_SSA = solve(ensembleprob, Tsit5(), trajectories=10000);

In [ ]:
using SciMLBase.EnsembleAnalysis

means_SSA = timeseries_steps_mean(sol_SSA)
plot!(means_SSA.t, [means_SSA[1,:], means_SSA[2,:]], lw=1.5, labels=["SSA μ₁₀" "SSA μ₀₁"], linestyle=:dash,
      linecolor=[1 2], background_color_legend=nothing, legend=:bottomright)

In [ ]:
#savefig("Brusselator_time-dependent_SSA.svg")